In [3]:
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import random

from pydub import AudioSegment
import noisereduce as nr

Add Gaussian Noise

In [4]:
def add_white_noise(signal,noise_factor):   # use a random noise factor
    noise = np.random.normal(0,signal.std(),signal.size)
    augmented_signal = signal + noise*noise_factor
    return augmented_signal

Time Stretching

In [5]:
def time_stretch(signal, stretch_rate):
    return librosa.effects.time_stretch(signal, rate = stretch_rate) #use stretch rate 0.8 to 1.2

Pitch Scalling 

In [6]:
def pitch_scale(signal,sr, num_semitones):
    return librosa.effects.pitch_shift(signal,sr =sr,n_steps= num_semitones) #use 4 to 5 semitones

Random Gain

In [7]:
def random_gain(signal,min_gain_factor,max_gain_factor):
    gain_factor = random.uniform(min_gain_factor,max_gain_factor)
    return signal*gain_factor

Denoise

In [8]:
def denoise(audio_input_path,audio_output_path):
    # Load audio file
    audio = AudioSegment.from_file(audio_input_path)

    # Convert audio to numpy array
    samples = np.array(audio.get_array_of_samples())

    # Reduce noise
    reduced_noise = nr.reduce_noise(samples, sr=audio.frame_rate)

    # Convert reduced noise signal back to audio
    reduced_audio = AudioSegment(
        reduced_noise.tobytes(), 
        frame_rate=audio.frame_rate, 
        sample_width=audio.sample_width, 
        channels=audio.channels
    )
    # Save reduced audio to file
    reduced_audio.export(audio_output_path+"_denoised.wav", format="wav")
    return 

By using Enrollment files , makes a new enrollment files by data augmentation

In [10]:
import os
from pathlib import Path

for i in os.listdir("C:/Users/nidul/Desktop/ACCA Folder My/Signal processing cup/ROBOVOX_SP_CUP_2024/single-channel/enrollment"):
    input_dir = os.path.join("C:/Users/nidul/Desktop/ACCA Folder My/Signal processing cup/ROBOVOX_SP_CUP_2024/single-channel/enrollment",i)  # give the location of the enrolment files here
    output_dir = os.path.join("C:/Users/nidul/Desktop/ACCA Folder My/Signal processing cup/ROBOVOX_SP_CUP_2024/single-channel/enrollment_augmented",i)   #give the loaction to create enrolment_augmented
    os.mkdir(output_dir)  
    files = [str(x) for x in list(Path(input_dir).glob("*.wav*"))]
    for file in files:
        signal,sr = librosa.load(file)

        augmented_white_noise_signal_1 = add_white_noise(signal,0.2)
        augmented_white_noise_signal_2 = add_white_noise(signal,0.3) 
        augmented_time_scalling_signal_1 = time_stretch(signal,1.2)
        augmented_time_scalling_signal_2 = time_stretch(signal,0.9)
        augmented_pitch_scalling_signal = pitch_scale(signal,sr,0.2) 
        augmented_random_gain_signal_1 = random_gain(signal,3,6)
        augmented_random_gain_signal_2 = random_gain(signal,4,8)


        base_name = os.path.basename(file)
        file_name, file_extension = os.path.splitext(base_name)

        sub_output_dir = os.path.join("C:/Users/nidul/Desktop/ACCA Folder My/Signal processing cup/ROBOVOX_SP_CUP_2024/single-channel/enrollment_augmented",i,file_name) 

        sf.write(sub_output_dir+".wav",signal,sr)
        # denoise(file,sub_output_dir)
        sf.write(sub_output_dir+"_with_white_noise_1.wav",augmented_white_noise_signal_1,sr)
        sf.write(sub_output_dir+"_with_white_noise_2.wav",augmented_white_noise_signal_2,sr)
        sf.write(sub_output_dir+"_with_time_scalling_1.wav",augmented_time_scalling_signal_1,sr)
        sf.write(sub_output_dir+"_with_time_scalling_2.wav",augmented_time_scalling_signal_2,sr)
        sf.write(sub_output_dir+"_with_pitch_scalling.wav",augmented_pitch_scalling_signal,sr)
        sf.write(sub_output_dir+"_with_random_gain_1.wav",augmented_random_gain_signal_1,sr)
        sf.write(sub_output_dir+"_with_random_gain_2.wav",augmented_random_gain_signal_2,sr)